In [19]:
import pandas as pd
import numpy as np
import os
import pickle
import networkx as nx
import mygene
import torch

In [4]:
current_directory = os.getcwd()

somepath = os.path.abspath(
    os.path.join(current_directory, '..', 'Data', 'RNA_dataset_tabular_R3.csv'))

# expression data
tabular_data = pd.read_csv(somepath, sep = ',', index_col = 0)
tabular_data

,NFKB1,TNIP2,AMOT,VASP,SS18L1,SMARCA4,SMURF1,HSPA5,SKIL,UBE2I,...,GALNT13,SLC17A1,MGST2,CHPT1,STK17A,SULF2,CD276,TIPARP,WNT16,PFS
id,,,,,,,,,,,,,,,,,,,,,
X00936b9285d6b8665ae9122993fb8e91,6.10,4.38,3.07,6.14,4.70,7.52,4.93,8.37,6.21,5.69,...,0.01,4.47,7.07,5.23,4.08,8.68,7.09,4.94,0.01,4.172485
X105622fadc33f23755ac2df823110aca,5.07,3.33,1.73,5.11,5.62,6.58,4.34,7.42,6.15,4.86,...,1.07,4.45,7.40,6.84,2.84,6.08,6.56,4.61,0.01,16.591376
Xe44f39747a8e84b02b4cb24659312144,6.13,4.41,3.23,6.32,5.57,8.02,5.14,7.55,6.87,6.27,...,2.13,2.94,6.66,5.81,4.16,6.33,7.14,8.42,0.01,11.104723
X293dd1284496215e9a0eca9f17a98e7e,5.82,4.30,3.44,6.45,4.86,7.45,4.90,8.39,6.83,5.70,...,1.60,0.83,6.67,5.27,4.12,6.97,6.73,6.28,0.01,14.028747
X01ed7190ce00862696edbf047b542045,6.15,4.21,3.90,5.93,4.43,7.60,4.74,8.31,6.38,5.94,...,0.01,6.70,6.53,5.84,4.35,5.67,6.66,4.93,0.01,12.418891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X50772aa64efb859960b20f8801cd6f58,6.27,3.78,3.98,5.95,4.66,7.18,4.61,8.04,6.62,5.80,...,2.31,7.81,7.26,6.15,3.52,5.92,6.74,5.48,0.01,4.271047
X91bcd3067a1a7954692d836515e04869,6.12,3.94,3.25,6.09,4.98,7.32,5.02,8.21,6.71,5.89,...,0.01,0.01,6.38,5.20,4.05,7.84,7.59,7.59,0.01,2.496920
Xc7439a06ffa32b313b0ec1b987b992a2,5.91,3.62,3.34,5.80,8.86,8.69,4.52,8.15,5.77,6.85,...,0.01,7.31,6.76,7.12,3.25,6.81,6.82,5.27,0.01,6.505133


In [7]:
print(set(tabular_data.dtypes))

{dtype('float64')}


In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()

X = tabular_data.drop(columns=['PFS'])
Y = tabular_data['PFS']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns, index = X_train.index)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns, index = X_test.index)

Y_train

id
Xf7ee4a331a8fd70edc240dff978c170d    13.897331
X5cabb9f0afcc6ec89f28a72e66b75c5b     3.909651
X2069a43cf26725e339dae5a64490a17f     1.379877
X3cdb45cfe9ffaac83f36c75e4af759e6     3.745380
X8783adc097e910b52c11f50e2a4aff2a     9.626283
                                       ...    
Xe1b0f87c25889df31bc0fe76a68853b6    12.517454
X502dfff1180e1952d964196c8493633b     0.032854
Xfdb742505b30978af60b426c93a32f9e     5.585216
X5db2ba740f908fe3a3e6832d48949346     1.412731
X490b3aaf0b34d71712e894ca670bff00     7.030801
Name: PFS, Length: 580, dtype: float64

In [33]:
torch.tesnY_train.values

array([13.8973306 ,  3.90965092,  1.3798768 ,  3.74537988,  9.62628337,
       19.35112936,  6.89938398,  8.57494867,  1.60985626,  6.9650924 ,
        5.65092403,  4.23819302,  2.56262834,  8.41067762,  1.24845996,
        1.41273101,  7.16221766,  4.20533881,  6.93223819,  8.31211499,
        7.03080082,  8.3449692 ,  8.3449692 ,  6.93223819, 12.32032854,
        1.41273101,  0.49281314,  1.41273101, 11.30184805,  2.85831622,
        1.41273101,  8.24640657,  4.1724846 ,  6.93223819,  0.03285421,
        4.1724846 ,  6.89938398,  8.11498973, 13.83162218, 18.00410678,
       16.65708419,  2.85831622,  4.1724846 ,  1.41273101,  4.43531828,
        5.42094456, 10.84188912,  2.7926078 ,  1.41273101,  3.02258727,
        5.84804928,  6.76796715, 11.07186858,  8.04928131,  2.92402464,
        2.62833676, 14.06160164, 12.61601643,  2.7926078 , 12.25462012,
        5.5523614 ,  4.20533881,  6.93223819,  6.93223819,  8.31211499,
        8.31211499,  5.61806982, 12.58316222,  4.82956879,  4.13

In [30]:
dataobj = X_train.iloc[0]
dataobj = torch.tensor(dataobj.values)
dataobj.reshape(-1, len(dataobj))

tensor([[0.6465, 0.4970, 0.4626,  ..., 0.4338, 0.3231, 0.0000]],
       dtype=torch.float64)

In [7]:
current_directory = os.getcwd()

somepath = os.path.abspath(
    os.path.join(current_directory, '..', 'Data', 'RNA_dataset_graph_R3.pkl'))

with open(somepath, 'rb') as f:
    loaded_object = pickle.load(f)

In [8]:
loaded_object